In [1]:
import numpy as np
import os
import pickle
import pickle
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from transformers import RobertaTokenizer, RobertaForSequenceClassification


C:\Users\vsing\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the tokenizer and model of sentiment
tokenizer = RobertaTokenizer.from_pretrained('saved_model_sentiment')
model = RobertaForSequenceClassification.from_pretrained('saved_model_sentiment')

# Load the tokenizer and model for category
load_directory = 'save_model_category_2'  
loaded_model = load_model(os.path.join(load_directory, 'my_category_model.h5'))
with open(os.path.join(load_directory, 'tokenizer.pickle'), 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

In [3]:
# Define category mapping and sentiment labels
category_mapping = {0: 'ambience', 1: 'food', 2: 'miscellaneous', 3: 'price', 4: 'service'}
sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}


# Define a function to predict aspect category
def predict_category(sentence):
    max_sequence_length = 63  # Define your desired sequence length
    sentence = [sentence]  # Convert to a list for consistency with previous data
    sequences = loaded_tokenizer.texts_to_sequences(sentence)
    sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
    predicted_category_label = loaded_model.predict(sequences)
    predicted_category_label = np.argmax(predicted_category_label)
    predicted_category = category_mapping[predicted_category_label]
    return predicted_category

# Define the function to predict sentiment
def predict_sentiment(sentence):
    # Assuming you've already loaded your tokenizer and model for sentiment analysis
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        sentiment_logits = outputs.logits
        sentiment_probs = torch.softmax(sentiment_logits, dim=1)  # Apply softmax
    sentiment_index = torch.argmax(sentiment_probs, dim=1).item()
    predicted_sentiment_label = sentiment_labels[sentiment_index]
    return predicted_sentiment_label

In [4]:
# List of sentences for testing
sentences = [
    "The food was delicious and flavorful.",
    "The service was slow and disappointing.",
    "The ambience of the restaurant was cozy and inviting.",
    "I couldn't figure out what the item on the menu was.",
    "This restaurent is quite costly."
]

# Predict aspect category and sentiment for each sentence
for sentence in sentences:
    predicted_category = predict_category(sentence)
    predicted_sentiment = predict_sentiment(sentence)
    print(f'Sentence: "{sentence}"')
    print(f'Predicted Aspect Category: {predicted_category}')
    print(f'Predicted Sentiment: {predicted_sentiment}')
    print()

1/1 [==============================] - 3s 3s/step
Sentence: "The food was delicious and flavorful."
Predicted Aspect Category: food
Predicted Sentiment: positive

1/1 [==============================] - 0s 51ms/step
Sentence: "The service was slow and disappointing."
Predicted Aspect Category: service
Predicted Sentiment: negative

1/1 [==============================] - 0s 62ms/step
Sentence: "The ambience of the restaurant was cozy and inviting."
Predicted Aspect Category: ambience
Predicted Sentiment: positive

1/1 [==============================] - 0s 54ms/step
Sentence: "I couldn't figure out what the item on the menu was."
Predicted Aspect Category: food
Predicted Sentiment: negative

1/1 [==============================] - 0s 50ms/step
Sentence: "This restaurent is quite costly."
Predicted Aspect Category: miscellaneous
Predicted Sentiment: negative

